# H002: Numerical errors

Student 1: David Álvarez Saez

Student 2: Andrés Teruel Fernández

In [1]:
import sys
import math
import scipy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Exercise 1.

The precision and magnitude of the numbers that can be represented and manipulated in a computer depend on the environment in which the operations are performed. For real numbers, **double precision** is commonly used.

1. The goal of this exercise is to determine, using a systematic search procedure, the largest and the smallest positive (different from zero) values that can be represented in the following environments:
    1. Excel (without using *Visual Basic*).
    2. Python.
    
2. Provide an explanation of the results obtained.

<u>HINTS</u>: 
1. Use a *while* loop
2. Try to understand the information provided by `print(sys.float_info)`.

In [2]:
import sys
print(sys.float_info)

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)


In [3]:
# Compute the largest positive double precision value.
exp = 0
aux = 1.0

# Find the largest power of 10 that can be represented
while not np.isinf(aux):
    exp = exp + 1
    aux = aux * 10.0

# exp is 308 which is equal to sys.float_info.max_10.exp
exp = exp - 1
max_real = 0.0;

# Go digit by digit from 10**exp until 10**0 looking for the biggest integer which does not overflow
for i in range(exp, -1, -1):
    aux = max_real
    
    while not np.isinf(aux):
        max_real = aux
        aux = aux + (10.0**i)
        
        if max_real == aux:
            aux = np.inf

# The answer is returned in the variable max_real.
print(f"sys.float_info.max={sys.float_info.max}")
print(f"max_real={max_real}")

sys.float_info.max=1.7976931348623157e+308
max_real=1.7976931348623157e+308


In [4]:
# Compute the smallest positive double precision value. 
exp = 0
aux = 1.0

# Find the smallest power of 10 that can be represented
while aux != 0:
    exp = exp - 1
    aux = 10.0**exp
    
# exp is -323
exp = exp + 1
min_real = 10.0**(exp)

# 10**-324 underflows, but can we find a number i > 1 which makes i*10**-324 to not underflow?
for i in range (9, 0, -1):
    aux = (i*10**-1)*(10.0**exp)
    if aux != 0:
        min_real = aux

# The answer is stored in the variable min_real.
print(f"sys.float_info.min={sys.float_info.min}")
print(f"math.ulp(0.0)={math.ulp(0.0)}")
print(f"min_real={min_real}")

sys.float_info.min=2.2250738585072014e-308
math.ulp(0.0)=5e-324
min_real=5e-324


#### Explicación

- Para encontrar el positivo de doble precisión más grande que se pueda representar en Python, primero encontramos la potencia de 10 más grande que no haga *overflow* (en este caso es 10**308, como nos confirma *sys.float_info.max_10.exp*). Para finalizar iteramos por todos los 309 dígitos, de más a menos significativos, encontrando para cada dígito el mayor valor entre 0 y 9 que no haga *overflow*.

    En teoría podemos buscar solo entre el dígito 309 hasta el 292 ya que la mantisa está limitada a 16 dígitos en base 10 (pese a que *sys.float_info.dig* indica que son 15 los que se pueden representar fielmente, 16 es el máximo). Pese a este detalle, se iteran los 309 dígitos puesto que no supone una carga computacional alta.

    Cuando comparamos nuestro resultado con *sys.float_info.max_10.max* vemos que obtenemos el mismo valor.

- Para encontrar el positivo de doble precisión más pequeño que se pueda representar en Python, encontraremos la potencia de 10 más pequeña que no haga *underflow*. Después buscamos si hay algún valor mayor a 1 en la siguiente potencia más pequeña de 10 que no haga *underflow*.

    En este ejercicio encontramos un valor inesperado, ya que según *sys.float_info.min* el número positivo en coma flotante normalizado más pequeño es 2.2250738585072014e-308, pero nuestro resultado es min_real=5e-324 que claramente es más pequeño.

    Tras investigar un poco, encontramos que el número positivo en coma flotante no-normalizado más pequeño posible es math.ulp(0.0)=5e-324, confirmando nuestro resultado.

---

### Exercise 2.

Due to rounding errors, in double precision there is a quantity **sys.float_info.epsilon** such that 

```
(1.0 + sys.float_info.epsilon) == 1.0
# False (the values compared are different)
 
(1.0 + sys.float_info.epsilon/2.0) == 1.0
# True (the values compared are equal)
```
1. Determine the largest values of *small-values* such that

```
(1.0 + small_values[0]) == 1.0
# True (the values compared are equal)

(1048576.0 + small_values[1]) == 1048576.0
# True (the values compared are equal)

(0.0009765625 + small_values[2]) == 0.0009765625
# True (the values compared are equal)
```

2. Is there any relation among these values?
3. Provide an explanation of the results obtained.

In [5]:
def find_small_value(n: float, initial_exp: int) -> float:
    exp = initial_exp
    small_value = 10**exp
    
    while (small_value + n) != n:
        exp = exp - 1
        small_value = 10.0**exp
    
    aux = None
    
    while aux != small_value:
        aux = small_value
        i = 1
        
        while i < 10 and (aux + n) == n:
            small_value = aux
            aux = aux + (10.0**exp)
            i = i + 1
    
        exp = exp - 1

    return small_value

In [6]:
small_values = [None, None, None]

small_values[0] = find_small_value(1.0, 0)
small_values[1] = find_small_value(1048576.0, 0)
small_values[2] = find_small_value(0.0009765625, -10)

print((1.0 + small_values[0]) == 1.0)
print((1048576.0 + small_values[1]) == 1048576.0)
print((0.0009765625 + small_values[2]) == 0.0009765625)

True
True
True


####### YOUR EXPLANATION GOES HERE (markdown)

### Exercise 3.
Write a function to compute the factorial of a number
$$
n! = \prod_{i=1}^n i.
$$

In [7]:
def factorial(n: int) -> float:
    """ Compute the factorial of a number
    Args:
        n: Number whose factorial is computed

    Returns:
        n! in double precision (to avoid numerical problems).

    Examples:
        >>> factorial(0)
        1
        >>> factorial(5)
        120
        >>> factorial(100)
        9.33262154439441e+157
    """

    if n < 0:
        raise ValueError('Only positive integers allowed.')
    
    ret = 1.0
    
    for i in range(2, n+1):
        ret = ret * i

    return ret

In [8]:
print(factorial(0))
print(factorial(5))
print(factorial(100))

1.0
120.0
9.33262154439441e+157


#### Explicación

No hay necesidad de una explicación muy extensa, es una función muy simple que concatena la multiplicación de todos los números desde el 1 hasta n.

---

### Exercise 4.

A combinatorial number is the number of distinct manners in which one can select $k$ out of $n$ objects 
$$
 \binom{n}{k} = \frac{n!}{k!(n-k)!}.
$$

1. Design a function to compute combinatorial numbers based on the factorial function defined in the previous exercise.
2. Test the design by computing the combinatorial numbers $\binom{0}{0}, \binom{5}{2}, \binom{400}{0}, \binom{400}{400}, \binom{400}{200}$.
3. If the initial design fails in some of the example, explain the reason for this failure.
4. Modify the design so that the function yields the correct answer.


In [9]:
def combinatorial_number(n: int, k:int) -> float:
    """ Compute the factorial of a number
    Args:
        n: Number of objects for selection
        k: Number of selected objects

    Returns:
        The number of ways in which k objects can be selected from a set of n objects
        

    Examples:
        >>> combinatorial_number(0, 0)
        1.0
        >>> combinatorial_number(5, 0)
        1.0
        >>> combinatorial_number(5, 5)
        1.0
        >>> combinatorial_number(5, 2)
        10.0
        >>> combinatorial_number(400, 0)
        1.0
        >>> combinatorial_number(400, 400)
        1.0
        >>> combinatorial_number(400, 200)
        1.0295250013541435e+119
    """
    numerator = factorial(n)
    denominator = factorial(k) * factorial(n-k)

    ret = numerator / denominator
    
    return ret

In [10]:
print(combinatorial_number(0, 0))
print(combinatorial_number(5, 0))
print(combinatorial_number(5, 5))
print(combinatorial_number(5, 2))
print(combinatorial_number(400, 0))
print(combinatorial_number(400, 400))
print(combinatorial_number(400, 200))

1.0
1.0
1.0
10.0
nan
nan
nan


In [11]:
i = 0
while not np.isinf(factorial(i)):
    i += 1

print(f"{i - 1}! = {factorial(i-1)}")
print(f"{i}! = {factorial(i)}")

170! = 7.257415615307994e+306
171! = inf


#### Explicación

Los últimos tres ejemplos fallan debido a *overflows* en el numerador y denominador (400! es más grande que *max_real*, incluso 200! se sale del rango). Como enseña la celda previa, 170! es el factorial más grande que nuestra función factorial() puede computar.

La siguiente función se ha implementado para evitar *overflows*, y en consecuencia corregir el cálculo final de los resultados:

In [12]:
def combinatorial_number_2(n: int, k:int) -> float:
    """ Compute the factorial of a number
    Args:
        n: Number of objects for selection
        k: Number of selected objects

    Returns:
        The number of ways in which k objects can be selected from a set of n objects
        

    Examples:
        >>> combinatorial_number_2(0, 0)
        1.0
        >>> combinatorial_number_2(5, 0)
        1.0
        >>> combinatorial_number_2(5, 5)
        1.0
        >>> combinatorial_number_2(5, 2)
        10.0
        >>> combinatorial_number_2(400, 0)
        1.0
        >>> combinatorial_number_2(400, 400)
        1.0
        >>> combinatorial_number_2(400, 200)
        1.0295250013541435e+119
    """
    ret = 1.0
    numerator_factors = []
    denominator_factors = []

    # To avoid a big chunck of the computation, we will cancell the biggest
    # factor on the denominator (either k! or (n-k)!) with its corresponding part
    # on the numerator since k < n and n-k < n then if k >= n-k, n!/k! = 
    # n * (n-1) * ... * (k+1) will result on the smallest number being computed
    # however if n-k > k then n!/(n-k)! = n * (n-1) * ... * (n-k+1) will result in
    # a smaller intermediate value
    if k >= n - k:
        numerator_factors = [i for i in range(k + 1, n + 1)] 
        denominator_factors = [i for i in range(1, n - k + 1)]

    else:
        numerator_factors = [i for i in range(n - k + 1, n + 1)] 
        denominator_factors = [i for i in range(1, k + 1)]

    # After that, to avoid overflows while separately calculating the numerator and
    # denominator we will divide one by one the biggest factors of the numerator with the biggest 
    # factors of the denominator and multiply all the results together to form the final result.
    # At this point the number of factors in the numerator and denominator is the same
    for i in range(0, len(denominator_factors)):
        numerator_index = len(numerator_factors) - i - 1
        denominator_index = len(denominator_factors) - i - 1

        ret *= numerator_factors[numerator_index] / denominator_factors[denominator_index]
    
    return ret

In [13]:
print(combinatorial_number_2(0, 0))
print(combinatorial_number_2(5, 0))
print(combinatorial_number_2(5, 5))
print(combinatorial_number_2(5, 2))
print(combinatorial_number_2(400, 0))
print(combinatorial_number_2(400, 400))
print(combinatorial_number_2(400, 200))

1.0
1.0
1.0
10.0
1.0
1.0
1.029525001354145e+119


#### Explicación

Para evitar *overflows*, aplicamos dos tretas:

Primero cancelamos el factor más grande en el denominador con parte de los factores del numerador:

* Si k es mayor o igual que n-k:
$$
 \frac{n!}{k!(n-k)!} = \frac{n!}{k!}\frac{1}{(n-k)!} = \frac{\prod_{i=k+1}^n i}{(n-k)!}
$$

* Si n-k es mayor que k:
$$
 \frac{n!}{k!(n-k)!} = \frac{n!}{(n-k)!}\frac{1}{k!} = \frac{\prod_{i=n-k+1}^n i}{k!}
$$

Para finalizar resolvemos el principal problema proveniente de calcular por separado el numerador y el denominador, cuyos valores sueltos son mucho más grandes que el resultado final, provocando *overflows* antes de computar el resultado final.

Con el objetivo de minimizar los valores intermedios, computamos la solución de la siguiente manera:

* Si k es mayor o igual que n-k:
$$
 \frac{\prod_{i=k+1}^n i}{(n-k)!} = \prod_{i=1}^{n-k} \frac{k + i}{i}
$$

* Si n-k es mayor que k:
$$
 \frac{\prod_{i=n-k+1}^n i}{k!} = \prod_{i=1}^k \frac{i + n - k}{i}
$$

---

### Exercise 5.

Consider the following series 
$$  S = \sum_{n=1}^{\infty} \frac{1}{n^4} = \frac{\pi^4}{90}$$

We will attempt to estimate its value in an approximate manner with the help of a computer.

To this end, we compute the value of the truncatd series 
$$
S\approx S_N  = \sum_{n=1}^N \frac{1}{n^4},
$$
which provides an accurate approximation of $S$ for sufficiently large $N$
$$
\lim_{N \rightarrow \infty} S_N  = S.
$$

The approximation errors are defined as
$$
error_{abs}(N) = \left| S_N - S \right|; \\ 
error_{rel}(N) = \frac{\left| S_N - S \right|}{\left| S \right|} = \left| \frac{S_N}{S} - 1 \right|.
$$

The question is what is the minimum error that can be achieved and how large does $N$ have to be to achieve is minimum. 

To investigate this issue, we will consider two possible implementations of the computation 

<u> IMPLEMENTATION 1</u>: Forward sum

$$
S_N^{\rightarrow} = \left(\left(\left(1 + \frac{1}{2^4} \right)
+ \frac{1}{3^4} \right) + \ldots + \frac{1}{(N-1)^4} \right) + \frac{1}{N^4}.
$$	

<u> IMPLEMENTATION 2</u>: backward sum
$$
S_N^{\leftarrow} = \left(\left(\left(
\frac{1}{N^4} +  \frac{1}{(N-1)^4} \right) + \frac{1}{(N-2)^4} \right)+⋯+ \frac{1}{2^4} \right) + 1.
$$	

1. Implement functions to compute these quantities.
2. Analyze the behavior of these quantities as a function of $N$.
    1. Does the error decrease as $N$ becomes larger? 
    2. What is the smallest value of the error, and for which value of $N$ is it attained?
    3. Are the smallest errors for $S_N^{\rightarrow} =$ and $S_N^{\leftarrow}$ equal? If not, which of the procedures is more accurate?
    4. Make a table (for instance, using [pandas](https://pandas.pydata.org/docs/user_guide/10min.html)) and a plot (using [matplotlib](https://matplotlib.org/stable/tutorials/index.html)) of the relative value of the approximation error of $S_N^{\rightarrow} =$ and $S_N^{\leftarrow}$ as a function of $N$.
3. Provide an explanation of the results obtained.

<u> HINT</u>: The explanation of the results hinges on the observations made in exercise 2.

In [14]:
# Exact sum 
S_exact = np.pi**4 / 90.0
print(S_exact)

1.082323233711138


In [15]:
# Series term
def inverse_power(n, p):
    return 1.0 / np.double(n)**p 

# Example (vectorized)
N = 5
n = np.arange(1, N+1)
print(inverse_power(n, p=4))

[1.         0.0625     0.01234568 0.00390625 0.0016    ]


In [16]:
# Forward sum

def forward_sum(series_term, N):
    # YOUR CODE GOES HERE
    

# Example of use

print(S_exact)

N = 1000
series_term = lambda n: inverse_power(n, p=4) 

S_N_forward = forward_sum(series_term, N)
print(S_N_forward)

IndentationError: expected an indented block after function definition on line 3 (872358748.py, line 9)

In [ ]:
# Backward sum

def backward_sum(series_term, N):
    # YOUR CODE GOES HERE

# Example of use

print(S_exact)

N = 1000
series_term = lambda n: inverse_power(n, p=4) 

S_N_backward = backward_sum(series_term, N)
print(S_N_backward)

In [ ]:
# YOUR CODE GOES HERE

####### YOUR EXPLANATION GOES HERE (markdown)

### Exercise 6.

Consider the following code:

In [ ]:
from scipy.integrate import quad
from scipy.stats import norm


mu = 1.35
sigma = 0.25
f  = lambda x: norm.pdf(x,mu,sigma); 

def mystery_function(alpha):
    x_inf = mu-alpha*sigma;
    x_sup = mu+alpha*sigma;
    return quad(f, x_inf, x_sup, epsabs=1.0e-10, epsrel=1.0e-10)

alpha = 2.0
print(mystery_function(alpha))

1. Explain what is being computed with this piece of code.
2. Does the value that is beign computed change with the values of $\mu$ and $\sigma$? 
3. Provide a mathematical derivation (use [latex within a markdown cell](https://towardsdatascience.com/write-markdown-latex-in-the-jupyter-notebook-10985edb91fd)) to explain what is observed.
4. Use the above derivation to compute the value of the integral for $\alpha = \infty$.
5. What is the smallest value of $\alpha$ that is needed to achieve a result numerically equivalent to the one obtained with $\alpha = \infty$ ?

<u>HINTS</u>: 
* Look for information on `quad`,`norm`. 
* You may need to modify the values of `epsabs` and / or `epsrel` to answer the last question.

In [ ]:
# YOUR CODE GOES HERE

####### YOUR EXPLANATION GOES HERE (markdown)

### Exercise 7.
Execute the following code several times.

In [ ]:
N = 4;
i = range(1, N+1)
i, j = np.meshgrid(i, i);
A = np.abs(i-j)
print(A)
v0 = np.random.randn(N)
MAX_ITER  = 50;
for i in range(MAX_ITER):
    v1 = A @ v0;
    squared_norm = np.dot(v1, v1)
    lamb =  squared_norm / np.dot(v0, v1)
    print(lamb)
    v0 = v1 / np.sqrt(squared_norm)

Provide an explanation of what the code does and of the results observed.

<u> HINT </u>:
Investigate the functionality of the numpy function `numpy.linalg.eig`.

In [ ]:
# YOUR CODE GOES HERE

####### YOUR EXPLANATION GOES HERE (markdown)

### Exercise 8.

1. Explain how can the eigenvectors and eigenvalues of a square matrix be used to compute its exponential.
2. Making use of the function `numpy.linalg.eig` compute the results of exponentiating Toeplitz' matrix
$$
A = \left(
\begin{array}{cccc}
0 & 1 & 2 & 3 \\
1 & 0 & 1 & 2 \\
2 & 1 & 0 & 1 \\
3 & 2 & 1 & 0 
\end{array} 
\right).
$$
3. Compare the results with those obtained with the scipy function `scipy.linalg.expm`

In [19]:
a = [[0, 1, 2, 3],
     [1, 0, 1, 2],
     [2, 1, 0, 1],
     [3, 2, 1, 0]]

# Compute the matrix eigenvalues and eigenvectors
eig_result = np.linalg.eig(a)

# Since 'a' is a matrix of 4x4 and it has 4 distinct eigenvalues
# 'a' it is diagonizable and its exponential can be computed fairly easy
s = eig_result.eigenvectors
s_inverse = np.linalg.inv(s)

d_exp = [[math.exp(eig_result.eigenvalues[0]), 0, 0, 0],
        [0, math.exp(eig_result.eigenvalues[1]), 0, 0],
        [0, 0, math.exp(eig_result.eigenvalues[2]), 0],
        [0, 0, 0, math.exp(eig_result.eigenvalues[3])]]

aux = np.dot(s, d_exp)
a_exp = np.dot(aux, s_inverse)
print(a_exp)

[[57.54897396 41.23414727 41.41932638 57.43936799]
 [41.23414727 30.18299519 29.70303099 41.41932638]
 [41.41932638 29.70303099 30.18299519 41.23414727]
 [57.43936799 41.41932638 41.23414727 57.54897396]]


In [22]:
a = [[0, 1, 2, 3],
     [1, 0, 1, 2],
     [2, 1, 0, 1],
     [3, 2, 1, 0]]

sp.linalg.expm(a)

array([[57.54897396, 41.23414727, 41.41932638, 57.43936799],
       [41.23414727, 30.18299519, 29.70303099, 41.41932638],
       [41.41932638, 29.70303099, 30.18299519, 41.23414727],
       [57.43936799, 41.41932638, 41.23414727, 57.54897396]])

#### Explicación

##### 1. Cálculo de la exponencial de la matriz A.

El cálculo de la exponencial de una matriz cuadrada es relativamente sencillo mediante el uso de autovalores y autovectores en el caso de que dicha matriz sea diagonalizable.

Por suerte para nosotros la matriz de Toeplitz es diagonalizable. Esto se confirma puesto que es una matriz de 4x4 y tiene 4 autovectores distintos.

1. Calculamos la matriz S cuyas columnas son los autovecores (v1, v2, v3 y v4) de A: 

$$
S = \left(
\begin{array}{cccc}
v1 & v2 & v3 & v4 
\end{array} 
\right).
$$

2. Mediante S, su inversa y los autovalores (lambda1, lambda2, lambda3 y lambda4), se puede calcular el exponencial de A de la siguiente manera:

$$
e^A = S \left(
\begin{array}{cccc}
e^{\lambda_{1}} & 0 & 0 & 0 \\
0 & e^{\lambda_{2}} & 0 & 0 \\
0 & 0 & e^{\lambda_{3}} & 0 \\
0 & 0 & 0 & e^{\lambda_{4}} 
\end{array} 
\right) S^{-1}
$$


##### 2. Comparación de resultados.

Siguiendo el algoritmo descrito en el apartado anterior y sp.linalg.expm() se obtiene el mismo resultado.

---